In [0]:
# Databricks Notebook: Bronze ingest from Apyflux Binance Futures Leaderboard (positions)
# Multi-account version (supports multiple encryptedUid → labeled as account_label)

import json, time, datetime as dt, hashlib
from typing import Dict, List, Tuple

import requests
from pyspark.sql import Row
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, LongType, BooleanType
)
from delta.tables import DeltaTable

# =========================
# (A) 실행/프로젝트 설정
# =========================
MODE  = "once"          # once | poll | forever
POLL_SECONDS = 60
MAX_POLLS    = 120
UPSERT_UPDATE_INGEST_TIME = True

CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
TABLE   = f"{CATALOG}.{SCHEMA}.bronze_futures_leaderboard_positions"

# 차트/캔들 조인 시 사용할 실버 테이블 명(필요 시 환경에 맞게 수정)
SILVER_CHARTS = f"{CATALOG}.{SCHEMA}.silver_charts"

BASE_URL       = "https://gateway.apyflux.com"
PATH_POS       = "/v1/getOtherPosition"

# ▶ 여러 계정(UID → 라벨): 대시보드 필터를 위해 라벨로도 저장
ENCRYPTED_UIDS: Dict[str, str] = {
    "14EA12E7412DC5A21DFF5E7EAC6013B9": "ILLIT",
    "38AF58C71A07AC6BDA18B8D97E7C3B00": "Geruntis",
    "1A7A15511721C96C03D029D4BA0AA64F": "Anonymous_A",
    "8E3794B912C699234158A15D3C9B9610": "Anonymous_B",
}

# Databricks Secrets
HEADERS = {
    "x-app-id":    dbutils.secrets.get("apyflux", "x_app_id"),
    "x-client-id": dbutils.secrets.get("apyflux", "x_client_id"),
    "x-api-key":   dbutils.secrets.get("apyflux", "x_api_key"),
    "Content-Type":"application/json",
}

# 스키마 자동 병합: 새 컬럼 추가 시 안전
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# =========================
# (B) 테이블 준비
# =========================
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA  IF NOT EXISTS {CATALOG}.{SCHEMA}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE} (
  source            STRING,     -- "apyflux.binance.futures.leaderboard"
  endpoint_name     STRING,     -- "getOtherPosition"
  uid               STRING,     -- encryptedUid
  account_label     STRING,     -- 계정 식별/필터용 라벨
  symbol            STRING,     -- 포지션 심볼
  entryPrice        DOUBLE,
  markPrice         DOUBLE,
  pnl               DOUBLE,
  roe               DOUBLE,
  amount            DOUBLE,
  leverage          DOUBLE,
  yellow            BOOLEAN,
  tradeBefore       BOOLEAN,
  update_ts         LONG,       -- 이벤트 시각(ms)
  event_time        TIMESTAMP,  -- update_ts 기반
  ingest_time       TIMESTAMP,  -- 적재 시각
  unique_key        STRING,
  raw_json          STRING,     -- 원본 JSON
  api_endpoint      STRING,     -- 호출 경로
  api_params_hash   STRING,     -- 요청 파라미터 해시
  dt                DATE
) USING DELTA
PARTITIONED BY (dt)
""")

# =========================
# (C) 유틸리티
# =========================
def params_hash(params: Dict) -> str:
    payload = json.dumps(params, sort_keys=True, separators=(",", ":"))
    return hashlib.sha256(payload.encode("utf-8")).hexdigest()

def _to_ms(d: dt.datetime) -> int:
    if d.tzinfo is None:
        d = d.replace(tzinfo=dt.timezone.utc)
    return int(d.timestamp() * 1000)

def _parse_update_ms(item: Dict, now_ms: int) -> int:
    """
    updateTimeStamp(ms) 또는 updateTime([Y,M,D,h,m,s,ns])를 ms로 변환.
    둘 다 없으면 now_ms.
    """
    uts = item.get("updateTimeStamp")
    if isinstance(uts, (int, float, str)):
        try:
            return int(uts)
        except Exception:
            pass

    ut = item.get("updateTime")
    if isinstance(ut, list) and len(ut) >= 6:
        Y, M, D, h, m, s = [int(x) for x in ut[:6]]
        ns = int(ut[6]) if len(ut) >= 7 else 0
        ms_part = ns / 1_000_000
        dt_utc = dt.datetime(Y, M, D, h, m, s, int(ms_part * 1000), tzinfo=dt.timezone.utc)
        return _to_ms(dt_utc)

    return now_ms

def _fetch_positions(enc_uid: str) -> Tuple[List[Dict], Dict[str,str], Dict[str,str], str]:
    """Apyflux getOtherPosition → (rows, headers, params, endpoint_path)"""
    url    = f"{BASE_URL}{PATH_POS}"
    params = {"encryptedUid": enc_uid}
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    payload = r.json()
    data = payload.get("data", {}) if isinstance(payload, dict) else {}
    rows = data.get("otherPositionRetList", []) or []
    return rows, dict(r.headers), params, PATH_POS

def _rows_to_bronze(enc_uid: str, account_label: str, rows: List[Dict], endpoint: str, params: Dict) -> int:
    """positions rows(list[dict]) → Bronze Delta UPSERT(idempotent) for one account"""
    if not rows:
        print(f"[INFO] no rows returned for uid={enc_uid}")
        return 0

    now = dt.datetime.now(dt.timezone.utc)
    now_iso = now.strftime("%Y-%m-%d %H:%M:%S")
    now_ms  = _to_ms(now)
    p_hash  = params_hash(params)

    recs = []
    for item in rows:
        sym    = item.get("symbol")
        upd_ms = _parse_update_ms(item, now_ms)
        ev     = dt.datetime.fromtimestamp(upd_ms/1000, tz=dt.timezone.utc)

        recs.append({
            "source":          "apyflux.binance.futures.leaderboard",
            "endpoint_name":   "getOtherPosition",
            "uid":             enc_uid,
            "account_label":   account_label,
            "symbol":          sym,
            "entryPrice":      float(item["entryPrice"]) if item.get("entryPrice") is not None else None,
            "markPrice":       float(item["markPrice"])  if item.get("markPrice")  is not None else None,
            "pnl":             float(item["pnl"])        if item.get("pnl")        is not None else None,
            "roe":             float(item["roe"])        if item.get("roe")        is not None else None,
            "amount":          float(item["amount"])     if item.get("amount")     is not None else None,
            "leverage":        float(item["leverage"])   if item.get("leverage")   is not None else None,
            "yellow":          bool(item["yellow"])      if item.get("yellow")     is not None else None,
            "tradeBefore":     bool(item["tradeBefore"]) if item.get("tradeBefore")is not None else None,
            "update_ts":       int(upd_ms),
            "event_time":      ev.strftime("%Y-%m-%d %H:%M:%S"),
            "ingest_time":     now_iso,
            "unique_key":      f"pos|{enc_uid}|{sym}|{upd_ms}",
            "raw_json":        json.dumps(item, separators=(",", ":")),
            "api_endpoint":    endpoint,
            "api_params_hash": p_hash,
            "dt":              ev.date().isoformat(),
        })

    schema = StructType([
        StructField("source",          StringType(), True),
        StructField("endpoint_name",   StringType(), True),
        StructField("uid",             StringType(), True),
        StructField("account_label",   StringType(), True),
        StructField("symbol",          StringType(), True),
        StructField("entryPrice",      DoubleType(), True),
        StructField("markPrice",       DoubleType(), True),
        StructField("pnl",             DoubleType(), True),
        StructField("roe",             DoubleType(), True),
        StructField("amount",          DoubleType(), True),
        StructField("leverage",        DoubleType(), True),
        StructField("yellow",          BooleanType(), True),
        StructField("tradeBefore",     BooleanType(), True),
        StructField("update_ts",       LongType(),   True),
        StructField("event_time",      StringType(), True),
        StructField("ingest_time",     StringType(), True),
        StructField("unique_key",      StringType(), True),
        StructField("raw_json",        StringType(), True),
        StructField("api_endpoint",    StringType(), True),
        StructField("api_params_hash", StringType(), True),
        StructField("dt",              StringType(), True),
    ])

    df = (spark.createDataFrame([Row(**r) for r in recs], schema)
            .withColumn("event_time",  to_timestamp(col("event_time")))
            .withColumn("ingest_time", to_timestamp(col("ingest_time")))
            .withColumn("dt",          col("dt").cast("date"))
            .dropDuplicates(["unique_key"])
         )

    delta_table = DeltaTable.forName(spark, TABLE)
    set_map = {
        "source":          "s.source",
        "endpoint_name":   "s.endpoint_name",
        "uid":             "s.uid",
        "account_label":   "s.account_label",
        "symbol":          "s.symbol",
        "entryPrice":      "s.entryPrice",
        "markPrice":       "s.markPrice",
        "pnl":             "s.pnl",
        "roe":             "s.roe",
        "amount":          "s.amount",
        "leverage":        "s.leverage",
        "yellow":          "s.yellow",
        "tradeBefore":     "s.tradeBefore",
        "update_ts":       "s.update_ts",
        "event_time":      "s.event_time",
        "raw_json":        "s.raw_json",
        "api_endpoint":    "s.api_endpoint",
        "api_params_hash": "s.api_params_hash",
        "dt":              "s.dt",
    }
    set_map["ingest_time"] = "s.ingest_time" if UPSERT_UPDATE_INGEST_TIME else "t.ingest_time"

    (delta_table.alias("t")
        .merge(df.alias("s"), "t.unique_key = s.unique_key AND t.dt = s.dt")
        .whenMatchedUpdate(set=set_map)
        .whenNotMatchedInsertAll()
        .execute())

    return df.count()

def ingest_for_account(enc_uid: str, label: str) -> int:
    rows, headers, params, ep = _fetch_positions(enc_uid)
    cnt = _rows_to_bronze(enc_uid, label, rows, ep, params)
    rem = headers.get("X-RateLimit-Remaining") or headers.get("x-ratelimit-remaining")
    if rem is not None:
        print(f"[Apyflux] remaining: {rem} (uid={enc_uid})")
    print(f"[Bronze LB] +{cnt} rows (uid={enc_uid}, label={label})")
    return cnt

def ingest_many(uid_map: Dict[str, str]) -> int:
    total = 0
    for uid, label in uid_map.items():
        try:
            total += ingest_for_account(uid, label)
        except Exception as e:
            print(f"[WARN] uid={uid} ({label}) failed: {e}")
            time.sleep(2)
    return total

# =========================
# (D) MAIN
# =========================
if MODE == "once":
    ingest_many(ENCRYPTED_UIDS)
    dbutils.notebook.exit("leaderboard positions once done")

elif MODE == "poll":
    for _ in range(MAX_POLLS):
        ingest_many(ENCRYPTED_UIDS)
        time.sleep(POLL_SECONDS)
    dbutils.notebook.exit("leaderboard positions poll done")

else:  # forever
    print(f"[LIVE] polling every {POLL_SECONDS}s for {len(ENCRYPTED_UIDS)} accounts")
    while True:
        try:
            ingest_many(ENCRYPTED_UIDS)
        except Exception as exc:
            print(f"[WARN] {exc}")
            time.sleep(5)
        time.sleep(POLL_SECONDS)
